In [10]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from pcpca import PCPCA
from sklearn.metrics import silhouette_score
import scanpy as sc
import os
from scripts import constants

adata = sc.read_h5ad(os.path.join(constants.DEFAULT_DATA_PATH, "zheng_2017/preprocessed/adata_top_2000_genes.h5ad"))


target_adata = adata[adata.obs['condition'] != 'healthy'].copy()
background_adata = adata[adata.obs['condition'] == 'healthy'].copy()

background_data = StandardScaler().fit_transform(adata[adata.obs['condition'] == 'healthy'].layers['count'])
target_data = StandardScaler().fit_transform(adata[adata.obs['patient_id'] == 'aml027'].layers['count'])

In [14]:
n, m = target_data.shape[1], background_data.shape[1]
model = PCPCA(n_components=2, gamma=0.5)
model.fit(target_data.transpose(), background_data.transpose())

In [15]:
PCPCA_salient_representations = model.transform(
    target_adata.X.transpose(),
    background_adata.X.transpose()
)[0].transpose()

In [16]:
silhouette_score(
    PCPCA_salient_representations,
    LabelEncoder().fit_transform(target_adata.obs['condition'])
)

0.15913859012623038